In [ ]:
!pip install salesforce-merlion
!git clone https://github.com/salesforce/Merlion.git
!pip install Merlion/ts_datasets/

fatal: destination path 'Merlion' already exists and is not an empty directory.
Processing ./Merlion/ts_datasets
  Preparing metadata (setup.py) ... done
  Created wheel for ts-datasets: filename=ts_datasets-0.1.0-py3-none-any.whl size=28760 sha256=d723f0330bc7a5d5cedbaaf35283dc3ef4f93355df1c00429eb428375adcd7b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-5453uppm/wheels/2c/b6/40/b02460ead944003f35edca35c96d734ab4e66b5ea0b1f08e68
Successfully built ts-datasets
  Attempting uninstall: ts-datasets
    Found existing installation: ts-datasets 0.1.0
    Uninstalling ts-datasets-0.1.0:
      Successfully uninstalled ts-datasets-0.1.0


In [ ]:
from merlion.utils import TimeSeries
from ts_datasets.anomaly import MSL

time_series, metadata = MSL()[0]
train_data = TimeSeries.from_pd(time_series[metadata.trainval])
test_data = TimeSeries.from_pd(time_series[~metadata.trainval])
test_labels = TimeSeries.from_pd(metadata.anomaly[~metadata.trainval])

print(f"Time series is {train_data.dim}-dimensional")

INFO:ts_datasets.anomaly.msl:Downloading https://www.dropbox.com/s/uv9ojw353qwzqht/SMAP.tar.gz?dl=1


Uncompressing /usr/local/lib/python3.10/data/smap/SMAP.tar.gz


INFO:ts_datasets.anomaly.msl:Uncompressing /usr/local/lib/python3.10/data/smap/SMAP.tar.gz


Preprocessing MSL


INFO:ts_datasets.anomaly.msl:Preprocessing MSL


Time series is 55-dimensional


In [ ]:
# We initialize models using the model factory in this tutorial
# We manually set the detection threshold to 2 (in standard deviation units) for all models
from merlion.models.factory import ModelFactory
from merlion.post_process.threshold import AggregateAlarms

#model1 = ModelFactory.create("ZMS",
                             #threshold=AggregateAlarms(alm_threshold=2))

model2 = ModelFactory.create("IsolationForest",
                             threshold=AggregateAlarms(alm_threshold=2))

model3 = ModelFactory.create("RandomCutForest",
                             threshold=AggregateAlarms(alm_threshold=2))

model4 = ModelFactory.create("AutoEncoder",
                             threshold=AggregateAlarms(alm_threshold=2))

model5 = ModelFactory.create("VAE",
                             threshold=AggregateAlarms(alm_threshold=2))

model6 = ModelFactory.create("DeepPointAnomalyDetector",
                             threshold=AggregateAlarms(alm_threshold=2))

for model in [model2, model3, model4, model5, model6]:
    print(f"Training {type(model).__name__}...")
    train_scores = model.train(train_data)

Training IsolationForest...
Training RandomCutForest...
Training AutoEncoder...
 |========================================| 100.0% Complete, Loss 0.0005
Training VAE...
 |========================================| 100.0% Complete, Loss 0.0006
Training DeepPointAnomalyDetector...


In [ ]:
from merlion.evaluate.anomaly import TSADMetric

for model in [model2, model3, model4, model5, model6]:
    labels = model.get_anomaly_label(test_data)
    precision = TSADMetric.PointAdjustedPrecision.value(ground_truth=test_labels, predict=labels)
    recall = TSADMetric.PointAdjustedRecall.value(ground_truth=test_labels, predict=labels)
    f1 = TSADMetric.PointAdjustedF1.value(ground_truth=test_labels, predict=labels)
    mttd = TSADMetric.MeanTimeToDetect.value(ground_truth=test_labels, predict=labels)
    print(f"{type(model).__name__}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1:        {f1:.4f}")
    print(f"MTTD:      {mttd}")
    print()

IsolationForest
Precision: 0.9638
Recall:    0.8192
F1:        0.8856
MTTD:      0 days 01:40:57

RandomCutForest
Precision: 0.9558
Recall:    0.9057
F1:        0.9301
MTTD:      0 days 00:56:02

AutoEncoder
Precision: 0.9643
Recall:    0.8696
F1:        0.9145
MTTD:      0 days 00:53:54

VAE
Precision: 0.9551
Recall:    0.8661
F1:        0.9084
MTTD:      0 days 01:00:40

DeepPointAnomalyDetector
Precision: 0.9543
Recall:    0.8305
F1:        0.8881
MTTD:      0 days 00:59:12



In [ ]:
from merlion.utils import TimeSeries
from ts_datasets.anomaly import SMAP

time_series2, metadata2 = SMAP()[0]
train_data2 = TimeSeries.from_pd(time_series2[metadata2.trainval])
test_data2 = TimeSeries.from_pd(time_series2[~metadata2.trainval])
test_labels2 = TimeSeries.from_pd(metadata2.anomaly[~metadata2.trainval])

print(f"Time series is {train_data2.dim}-dimensional")

Time series is 25-dimensional


In [ ]:
# We initialize models using the model factory in this tutorial
# We manually set the detection threshold to 2 (in standard deviation units) for all models
from merlion.models.factory import ModelFactory
from merlion.post_process.threshold import AggregateAlarms

#model7 = ModelFactory.create("ZMS",
                             #threshold=AggregateAlarms(alm_threshold=2))

model8 = ModelFactory.create("IsolationForest",
                             threshold=AggregateAlarms(alm_threshold=2))

model9 = ModelFactory.create("RandomCutForest",
                             threshold=AggregateAlarms(alm_threshold=2))

model10 = ModelFactory.create("AutoEncoder",
                             threshold=AggregateAlarms(alm_threshold=2))

model11 = ModelFactory.create("VAE",
                             threshold=AggregateAlarms(alm_threshold=2))

model12 = ModelFactory.create("DeepPointAnomalyDetector",
                             threshold=AggregateAlarms(alm_threshold=2))

for model in [model8, model9, model10, model11, model12]:
    print(f"Training {type(model).__name__}...")
    train_scores = model.train(train_data2)

Training IsolationForest...
Training RandomCutForest...
Training AutoEncoder...
 |========================================| 100.0% Complete, Loss 0.0004
Training VAE...
 |========================================| 100.0% Complete, Loss 0.0009
Training DeepPointAnomalyDetector...


In [ ]:
from merlion.evaluate.anomaly import TSADMetric
for model in [model8, model9, model10, model11, model12]:
    labels = model.get_anomaly_label(test_data2)
    precision = TSADMetric.PointAdjustedPrecision.value(ground_truth=test_labels2, predict=labels)
    recall = TSADMetric.PointAdjustedRecall.value(ground_truth=test_labels2, predict=labels)
    f1 = TSADMetric.PointAdjustedF1.value(ground_truth=test_labels2, predict=labels)
    mttd = TSADMetric.MeanTimeToDetect.value(ground_truth=test_labels2, predict=labels)
    print(f"{type(model).__name__}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1:        {f1:.4f}")
    print(f"MTTD:      {mttd}")
    print()

IsolationForest
Precision: 0.9579
Recall:    0.5201
F1:        0.6742
MTTD:      0 days 00:35:42

RandomCutForest
Precision: 0.9614
Recall:    0.6561
F1:        0.7799
MTTD:      0 days 01:37:26

AutoEncoder
Precision: 0.9577
Recall:    0.5257
F1:        0.6788
MTTD:      0 days 00:49:42

VAE
Precision: 0.9562
Recall:    0.5137
F1:        0.6683
MTTD:      0 days 00:47:53

DeepPointAnomalyDetector
Precision: 0.9617
Recall:    0.6305
F1:        0.7617
MTTD:      0 days 01:37:01

